In [2]:
from bs4 import BeautifulSoup
import requests
import time
import datetime as dt 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import geopandas

import pickle
import os

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

In [ ]:
def get_countries():
    countries_page='https://weatherspark.com/'
    html=requests.get(countries_page,headers=headers).content
    soup=BeautifulSoup(html,'html.parser')

    countries_urls_dict={}
    for i in soup.select('div[class="row"] li a'):
        country_name=i.text
        country_url='https://weatherspark.com'+i['href']
        if 'https://weatherspark.com/countries/US/GA' == str(country_url):
            country_name='Georgia (USA State)'
        elif 'https://weatherspark.com/countries/GE' == str(country_url):
            country_name='Georgia'

        countries_urls_dict[country_name]=country_url

    return countries_urls_dict

In [ ]:
countries_urls_dict=get_countries()

In [ ]:
all_cities={}

for index, (country_name, country_url) in enumerate(countries_urls_dict.items(),start=1):

    country_page=country_url
    html=requests.get(country_page,headers=headers).content
    soup=BeautifulSoup(html,'html.parser')

    cities_urls_dict={}
    try:
        for i in [i for i in soup.select('div[class="row"] div[class="col-lg-12"]') if 'Places' in i.text][-1].select('li a'):
            city_name=f'{i.text} ({country_name})'
            city_url='https://weatherspark.com'+i['href']

            cities_urls_dict[city_name]=city_url
    except:
        continue
    
    all_cities.update(cities_urls_dict)

    print(index,end='\r')

In [3]:
#pickle.dump(all_cities,open('all_cities.pickle','wb'))

In [ ]:
#get city coordinates

import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

def get_coordinates(city, country):
    geolocator = Nominatim(user_agent="your_app_name")
    try:
        location = geolocator.geocode(f"{city}, {country}")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        print("Error: geocode timed out")
        return None, None
    except GeocoderServiceError as e:
        print(f"Error: geocode failed with error {e}")
        return None, None


city_coordinates={}
for i,city in enumerate(all_cities,start=1):
    city_name=city.split('(')[0].strip()
    country_name=city.split('(')[-1].split(')')[0].strip()
    latitude, longitude = get_coordinates(city_name, country_name)
    city_coordinates[city]={'Lat':latitude,'Lon':longitude}

    if i%10==0:
        print(i,end='\r')

In [ ]:
#pickle.dump(city_coordinates,open('city_coordinates.pickle','wb'))